In [ ]:
import pandas as pd
import geopandas as gpd
import pydeck as pdk
import matplotlib.pyplot as plt
import numpy as np
import folium
from census import Census
from us import states

# Get a US Census API Key [here](https://api.census.gov/data/key_signup.html) 

Copy-paste your API Key when prompted when running the cell below.

In [ ]:
import os
from getpass import getpass

# Try to read from env first, otherwise prompt you
CENSUS_API_KEY = os.getenv("CENSUS_API_KEY") or getpass("Enter your US Census API key: ")

os.environ["CENSUS_API_KEY"] = CENSUS_API_KEY

print("Key loaded, length:", len(CENSUS_API_KEY), "characters")

# ACS Data

In [ ]:
inc_vars = [
    "B08119_018E" # Drove alone + >$75k income
]

In [ ]:
c = Census(CENSUS_API_KEY, year=2022)

# List of Bay-Area county FIPS:
bay_fips = ["001","013","041","055","075","081","085","095","097"]

In [ ]:
def get_inc_by_tract(inc_list, inc_var):
    for county in bay_fips:
        inc_list += c.acs5.state_county_tract(
            (inc_var,"NAME"),
            states.CA.fips,
            county,
            Census.ALL
        )

    df = pd.DataFrame(inc_list)
    df["GEOID"] = df.state + df.county + df.tract
    df = df[["GEOID",inc_var]]

    return df

In [ ]:
inc1 = []
df_inc1 = get_inc_by_tract(inc1, inc_vars[0])

In [ ]:
# Load Bay-Area tracts shapefile
path = "/Users/dsong/Library/CloudStorage/OneDrive-UniversityofIllinois-Urbana/Research/UROP 2025 - UAM/Demand Analysis/TIGER Line 2022 Tract/tl_2022_06_tract.shp"
tracts = gpd.read_file(path)[["GEOID","geometry"]]
bay_tracts = tracts[tracts.GEOID.str[:5].isin({"06001","06013","06041",
                                             "06055","06075","06081",
                                             "06085","06095","06097"})]

# Merge population → GeoDataFrame
gdf_acs = bay_tracts.merge(df_inc1, on="GEOID", how="left").fillna(0)
gdf_acs = gdf_acs.drop(columns="geometry")

# LODES Data

In [ ]:
data_var = "S000"

In [ ]:
# Read CSV file containing LODES data, update path to your local file
path = "/Users/dsong/Library/CloudStorage/OneDrive-UniversityofIllinois-Urbana/Research/UROP 2025 - UAM/Demand Analysis/LODES/ca_od_main_JT00_2022.csv"
od = pd.read_csv(path, dtype={"w_geocode": str, "h_geocode": str})

In [ ]:
# Extract 11-digit tract GEOIDs (tract codes, not full GEOIDs for blocks)
od["h_tract"] = od["h_geocode"].str[:11]
od["w_tract"] = od["w_geocode"].str[:11]

od_copy = od.copy()

# Choose a data variable to plot

In [ ]:
data_var = "S000"

In [ ]:
# Group by home→work tract pairs and sum jobs in data_var
od_tract = (
    od
    .groupby(["h_tract", "w_tract"], as_index=False)[data_var]
    .sum()
)

# Also compute total out-flow per origin tract for a choropleth
sum_H = (
    od_tract
    .groupby("h_tract", as_index=False)[data_var]
    .sum()
    .rename(columns={"h_tract": "GEOID", data_var: "sum_H"})
)

In [ ]:
# Remove intra-tract commutes
od_overlap = od_copy[od_copy["h_tract"] != od_copy["w_tract"]]
# Group by home→work tract pairs
od_tract_overlap = (od_overlap.groupby(["h_tract", "w_tract"])[[data_var]]
            .sum()
            .reset_index())

H2W = (
    od_tract_overlap
    .groupby("h_tract", as_index=False)[data_var]
    .sum()
    .rename(columns={"h_tract": "GEOID", data_var: "H2W"})
)

In [ ]:
# Read the TIGER/Line shapefile for California tracts, static choropleth
path = "/Users/dsong/Library/CloudStorage/OneDrive-UniversityofIllinois-Urbana/Research/UROP 2025 - UAM/Demand Analysis/TIGER Line 2022 Tract/tl_2022_06_tract.shp"
tracts = gpd.read_file(path)[["GEOID", "geometry"]]

# Merge the origin totals onto the GeoDataFrame
gdf_H2W = tracts.merge(H2W, on="GEOID", how="left").fillna(0)
gdf_sum_H = tracts.merge(sum_H, on="GEOID", how="left").fillna(0)

# Filter to Bay-Area tracts by county FIPS
bay_counties = {
    "06001",  # Alameda
    "06013",  # Contra Costa
    "06041",  # Marin
    "06055",  # Napa
    "06075",  # San Francisco
    "06081",  # San Mateo
    "06085",  # Santa Clara
    "06095",  # Solano
    "06097",  # Sonoma
}
# GEOID[:5] is state+county FIPS
gdf_H2W = gdf_H2W[gdf_sum_H["GEOID"].str[:5].isin(bay_counties)].copy()
gdf_sum_H = gdf_sum_H[gdf_sum_H["GEOID"].str[:5].isin(bay_counties)].copy()

gdf_H2W = gdf_H2W.drop(columns="geometry")
gdf = gdf_sum_H.merge(
    gdf_H2W, on="GEOID", how="left"
)
gdf = gdf.merge(gdf_acs, on="GEOID", how="left")
gdf["final"] = gdf["H2W"] / gdf["sum_H"] * gdf["B08119_018E"]
gdf

# Static Choropleth

Yellow indicates more people leaving their tract to go somewhere else (more outbound travel beyond their tract)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
gdf.plot(
    column="final",
    cmap="magma_r",
    legend=True,
    legend_kwds={
        "label": "Commutes Originating in Tract",
        "fmt": "{:.0f}",
    },
    linewidth=0.1,
    edgecolor="gray",
    ax=ax
)
ax.set_title("2022 Estimated Demand — Bay Area")
ax.axis("off")
plt.show()

In [ ]:
# Define your view over the Bay Area
geojson = gdf.__geo_interface__
coords = np.vstack(gdf.geometry.centroid.apply(lambda p: (p.y, p.x)))
view_state = pdk.ViewState(
    latitude=coords[:,0].mean(),
    longitude=coords[:,1].mean(),
    zoom=9,
    pitch=0
)

# Center map on Bay Area
center = [view_state.latitude, view_state.longitude]
m = folium.Map(location=center, zoom_start=9, tiles=None)

# Add Google Streets as your basemap
folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Streets",
    control=False
).add_to(m)

# Add choropleth layer with legend
folium.Choropleth(
    geo_data=geojson,
    data=gdf,
    columns=["GEOID", "final"],
    key_on="feature.properties.GEOID",
    fill_color="YlOrBr",
    bins=8,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Esimated Demand",
).add_to(m)

folium.LayerControl().add_to(m)
m